# Trip data
This script is used to download data regarding the `trips`. To request the data, you need an API key. This should be stored in a `.env` file.

**NOTE:** It's recommended to use a virtual environment to execute this notebook in. Generate one using the `create_jupyter_env.sh` script before running this file.

In [ ]:
# Install libraries if not already installed
!pip3 install pandas python-dotenv

In [ ]:
# Import libraries
import os
import pandas as pd
import requests, urllib.request, urllib.parse, urllib.error, base64
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set secret from environment
skey = os.environ.get('SKEY')
static_key = os.environ.get('STATIC_KEY')

# Check if the environment variables are set
if None in [skey, static_key]:
    print('Environment variables are not set. Do you have a .env file?')
else:
    print('Successfully imported environment variables.')

In [ ]:
# Set request headers
headers = {
    'Ocp-Apim-Subscription-Key': skey,
}

# Set request parameters
params = urllib.parse.urlencode({
    'canceled': False,
    'delay': False,
    'position': True,
    # 'vehicleid': False,
    # 'tripid': False,
})

api_url = 'https://api.delijn.be/gtfs/v3/realtime?json=true'

# Make the request
response = requests.get(api_url, headers=headers, params=params)

if response.status_code == 200:
    try:
        data = response.json()
        print('Successfully retrieved data.')
    except Exception as e:
        print('An error occurred:', e)
        print('Response:', response.text)
else:
    print('Failed to retrieve data! Status code:', response.status_code)

In [ ]:
# Flatten function
def flatten_trip_update(tu):
    u = tu['tripUpdate']
    t = u['trip']
    result = {
        'id': tu['id'],
        'tripId': t['tripId']
    }
    if 'startDate' in t:
        result['tripStartDate'] = t['startDate']
    if 'scheduleRelationship' in t:
        result['tripScheduleRelationship'] = t['scheduleRelationship']
    if 'stopTimeUpdate' in u:
        result['departureDelay'] = u['stopTimeUpdate'][0]['departure']['delay']
        if 'stopId' in u['stopTimeUpdate'][0]:
            result['departureStopId'] = u['stopTimeUpdate'][0]['stopId']
        if 'vehicle' in u:
            result['vehicle'] = u['vehicle']['id']
        if 'timestamp' in u:
            result['timestamp'] = u['timestamp']
    result['orig'] = tu
    return result

# Generate convert data to trips dataframe
triplist = [flatten_trip_update(tu) for tu in data['entity']]
tripsdf = pd.DataFrame(triplist)

In [ ]:
# Display the last 5 rows of the dataframe
vehicleUpdates = tripsdf[tripsdf.vehicle.notnull()]
vehicleUpdates.tail()

In [ ]:
static_header = {
    'Ocp-Apim-Subscription-Key': static_key,
}

static_url = 'https://api.delijn.be/gtfs/static/v3/gtfs_transit.zip'

# Download the static GTFS data to a zip file
# WARNING: This might take a while!
with requests.get(static_url, headers=static_header, stream=True) as r:
    r.raise_for_status()
    with open("gtfs_transit.zip", 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
        print('Downloaded static GTFS data to gtfs_transit.zip')
        f.close()
    r.close()